<a href="https://colab.research.google.com/github/Rishabh5752/Text_Attack_Exp/blob/main/BERT_base_uncased_AGNews.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install textattack

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 418.7/418.7 kB 12.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.1/61.1 kB 8.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 373.1/373.1 kB 42.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 769.7/769.7 kB 50.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 365.7/365.7 kB 26.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.2/7.2 MB 113.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 125.2/125.2 kB 19.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.6/3.6 MB 99.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.4/41.4 MB 19.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 97.5/97

In [ ]:
!pip install tensorflow-text

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.0/6.0 MB 48.2 MB/s eta 0:00:00


In [ ]:
import textattack
import transformers

textattack: Updating TextAttack package dependencies.
textattack: Downloading NLTK required packages.
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package omw to /root/nltk_data...
[nltk_data] Downloading package universal_tagset to /root/nltk_data...
[nltk_data]   Unzipping taggers/universal_tagset.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [ ]:
model = transformers.AutoModelForSequenceClassification.from_pretrained("textattack/bert-base-uncased-ag-news")
tokenizer = transformers.AutoTokenizer.from_pretrained("textattack/bert-base-uncased-ag-news")
model_wrapper = textattack.models.wrappers.HuggingFaceModelWrapper(model, tokenizer)

In [ ]:
# Construct our four components for `Attack`

from textattack.constraints.pre_transformation import RepeatModification, StopwordModification

from textattack.constraints.semantics import WordEmbeddingDistance

In [ ]:
goal_function = textattack.goal_functions.UntargetedClassification(model_wrapper)

constraints = [

    RepeatModification(),
    StopwordModification(),
    WordEmbeddingDistance(min_cos_sim=0.7)

]

textattack: Unknown if model of class <class 'transformers.models.bert.modeling_bert.BertForSequenceClassification'> compatible with goal function <class 'textattack.goal_functions.classification.untargeted_classification.UntargetedClassification'>.
textattack: Downloading https://textattack.s3.amazonaws.com/word_embeddings/paragramcf.
100%|██████████| 481M/481M [00:14<00:00, 33.7MB/s]
textattack: Unzipping file /root/.cache/textattack/tmpcy93u35e.zip to /root/.cache/textattack/word_embeddings/paragramcf.
textattack: Successfully saved word_embeddings/paragramcf to cache.


In [ ]:
transformation = textattack.transformations.word_swaps.word_swap_homoglyph_swap.WordSwapHomoglyphSwap()
search_method = textattack.search_methods.BeamSearch(beam_width=8)

In [ ]:
#attack = textattack.attack_recipes.deepwordbug_gao_2018.DeepWordBugGao2018.build(model_wrapper)
attack =  textattack.Attack(goal_function, constraints, transformation, search_method)

#datasets.load_dataset('emotion', split =['test'])
#dataset=datasets.load_dataset('sms_spam', split =['test'])
dataset = textattack.datasets.HuggingFaceDataset("ag_news", dataset_columns=(["text"],'label'))
#attack =  textattack.Attack(goal_function, constraints, transformation, search_method)

Generating train split:   0%|          | 0/120000 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/7600 [00:00<?, ? examples/s]

Dataset ag_news downloaded and prepared to /root/.cache/huggingface/datasets/ag_news/default/0.0.0/bc2bcb40336ace1a0374767fc29bb0296cdaf8a6da7298436239c54d79180548. Subsequent calls will reuse this data.


  0%|          | 0/2 [00:00<?, ?it/s]

textattack: Loading datasets dataset ag_news, split train.


In [14]:

attack_args = textattack.AttackArgs(

    num_examples=100,

    log_to_csv="TAagnews.csv",

    checkpoint_interval=2,

    checkpoint_dir="checkpoints",

    disable_stdout=True

)

attacker = textattack.Attacker(attack, dataset, attack_args)

attacker.attack_dataset()

textattack: Logging to CSV at path TAagnews.csv


Attack(
  (search_method): BeamSearch(
    (beam_width):  8
  )
  (goal_function):  UntargetedClassification
  (transformation):  WordSwapHomoglyphSwap
  (constraints): 
    (0): WordEmbeddingDistance(
        (embedding):  WordEmbedding
        (min_cos_sim):  0.7
        (cased):  False
        (include_unknown_words):  True
        (compare_against_original):  True
      )
    (1): RepeatModification
    (2): StopwordModification
  (is_black_box):  True
) 






  0%|          | 0/100 [00:00<?, ?it/s]


  1%|          | 1/100 [00:25<41:54, 25.40s/it]


[Succeeded / Failed / Skipped / Total] 1 / 0 / 0 / 1:   1%|          | 1/100 [00:25<41:56, 25.42s/it]


[Succeeded / Failed / Skipped / Total] 1 / 0 / 0 / 1:   2%|▏         | 2/100 [04:08<3:22:43, 124.12s/it]


[Succeeded / Failed / Skipped / Total] 2 / 0 / 0 / 2:   2%|▏         | 2/100 [04:08<3:22:44, 124.13s/it]textattack: Saving checkpoint under "checkpoints/1687335736463.ta.chkpt" at 2023-06-21 08:22:16 after 2 attacks.





[Succeeded / Failed / Skipped / Total] 2 / 0 / 0 / 2:   3%|▎         | 3/100 [05:56<3:12:09, 118.86s/it]


[Succeeded / Failed / Skipped / Total] 3 / 0 / 0 / 3:   3%|▎         | 3/100 [05:56<3:12:10, 118.87s/it]


[Succeeded / Failed / Skipped / Total] 3 / 0 / 0 / 3:   4%|▍         | 4/100 [06:50<2:44:05, 102.56s/it]


[Succeeded / Failed / Skipped / Total] 4 / 0 / 0 / 4:   4%|▍         | 4/100 [06:50<2:44:06, 102.56s/it]textattack: Saving checkpoint under "checkpoints/1687335898461.ta.chkpt" at 2023-06-21 08:24:58 after 4 attacks.





[Succeeded / Failed / Skipped / Total] 4 / 0 / 0 / 4:   5%|▌         | 5/100 [07:38<2:25:18, 91.78s/it] 


[Succeeded / Failed / Skipped / Total] 5 / 0 / 0 / 5:   5%|▌         | 5/100 [07:38<2:25:18, 91.78s/it]


[Succeeded / Failed / Skipped / Total] 5 / 0 / 0 / 5:   6%|▌         | 6/100 [10:17<2:41:19, 102.97s/it]


[Succeeded / Failed / Skipped / Total] 6 / 0 / 0 / 6:   6%|▌         | 6/100 [10:17<2:41:19, 102.98s/it]textattack: Saving checkpoint under "checkpoints/1687336106067.ta.chkpt" at 2023-06-21 08:28:26 after 6 attacks.


...




[Succeeded / Failed / Skipped / Total] 89 / 2 / 1 / 92:  93%|█████████▎| 93/100 [2:22:17<10:42, 91.80s/it]


[Succeeded / Failed / Skipped / Total] 90 / 2 / 1 / 93:  93%|█████████▎| 93/100 [2:22:17<10:42, 91.80s/it]


[Succeeded / Failed / Skipped / Total] 90 / 2 / 1 / 93:  94%|█████████▍| 94/100 [2:22:57<09:07, 91.25s/it]


[Succeeded / Failed / Skipped / Total] 91 / 2 / 1 / 94:  94%|█████████▍| 94/100 [2:22:57<09:07, 91.25s/it]textattack: Saving checkpoint under "checkpoints/1687344066106.ta.chkpt" at 2023-06-21 10:41:06 after 94 attacks.





[Succeeded / Failed / Skipped / Total] 91 / 2 / 1 / 94:  95%|█████████▌| 95/100 [2:35:35<08:11, 98.27s/it]


[Succeeded / Failed / Skipped / Total] 91 / 3 / 1 / 95:  95%|█████████▌| 95/100 [2:35:35<08:11, 98.27s/it]


[Succeeded / Failed / Skipped / Total] 91 / 3 / 1 / 95:  96%|█████████▌| 96/100 [2:40:31<06:41, 100.32s/it]


[Succeeded / Failed / Skipped / Total] 92 / 3 / 1 / 96:  96%|█████████▌| 96/100 [2:40:31<06:41, 100.32s/it]textattack: Saving checkpoint under "checkpoints/1687345119254.ta.chkpt" at 2023-06-21 10:58:39 after 96 attacks.





[Succeeded / Failed / Skipped / Total] 92 / 3 / 1 / 96:  97%|█████████▋| 97/100 [2:40:52<04:58, 99.51s/it] 


[Succeeded / Failed / Skipped / Total] 93 / 3 / 1 / 97:  97%|█████████▋| 97/100 [2:40:52<04:58, 99.51s/it]


[Succeeded / Failed / Skipped / Total] 93 / 3 / 1 / 97:  98%|█████████▊| 98/100 [2:41:53<03:18, 99.12s/it]


[Succeeded / Failed / Skipped / Total] 94 / 3 / 1 / 98:  98%|█████████▊| 98/100 [2:41:53<03:18, 99.12s/it]textattack: Saving checkpoint under "checkpoints/1687345201734.ta.chkpt" at 2023-06-21 11:00:01 after 98 attacks.





[Succeeded / Failed / Skipped / Total] 94 / 3 / 1 / 98:  99%|█████████▉| 99/100 [2:48:07<01:41, 101.90s/it]


[Succeeded / Failed / Skipped / Total] 95 / 3 / 1 / 99:  99%|█████████▉| 99/100 [2:48:07<01:41, 101.90s/it]


[Succeeded / Failed / Skipped / Total] 95 / 3 / 1 / 99: 100%|██████████| 100/100 [3:00:26<00:00, 108.26s/it]


[Succeeded / Failed / Skipped / Total] 96 / 3 / 1 / 100: 100%|██████████| 100/100 [3:00:26<00:00, 108.26s/it]textattack: Saving checkpoint under "checkpoints/1687346314387.ta.chkpt" at 2023-06-21 11:18:34 after 100 attacks.
[Succeeded / Failed / Skipped / Total] 96 / 3 / 1 / 100: 100%|██████████| 100/100 [3:00:26<00:00, 108.26s/it]





+-------------------------------+---------+
| Attack Results                |         |
+-------------------------------+---------+
| Number of successful attacks: | 96      |
| Number of failed attacks:     | 3       |
| Number of skipped attacks:    | 1       |
| Original accuracy:            | 99.0%   |
| Accuracy under attack:        | 3.0%    |
| Attack success rate:          | 96.97%  |
| Average perturbed word %:     | 10.76%  |
| Average num. words per input: | 37.75   |
| Avg num queries:              | 3153.25 |
+-------------------------------+---------+


In [15]:
from textattack import Attacker

attacker = Attacker(attack, dataset)
attack_results=attacker.attack_dataset()

Attack(
  (search_method): BeamSearch(
    (beam_width):  8
  )
  (goal_function):  UntargetedClassification
  (transformation):  WordSwapHomoglyphSwap
  (constraints): 
    (0): WordEmbeddingDistance(
        (embedding):  WordEmbedding
        (min_cos_sim):  0.7
        (cased):  False
        (include_unknown_words):  True
        (compare_against_original):  True
      )
    (1): RepeatModification
    (2): StopwordModification
  (is_black_box):  True
) 



[Succeeded / Failed / Skipped / Total] 1 / 0 / 0 / 1:  10%|█         | 1/10 [00:24<03:44, 24.94s/it]

--------------------------------------------- Result 1 ---------------------------------------------
[[Business (96%)]] --> [[World (93%)]]

Wall St. Bears Claw Back Into the Black (Reuters) Reuters - [[Short-sellers]], [[Wall]] Street's dwindling\band of ultra-cynics, are seeing green again.

Wall St. Bears Claw Back Into the Black (Reuters) Reuters - [[Short˗sellers]], [[Wɑll]] Street's dwindling\band of ultra-cynics, are seeing green again.




[Succeeded / Failed / Skipped / Total] 2 / 0 / 0 / 2:  20%|██        | 2/10 [04:07<16:29, 123.69s/it]

--------------------------------------------- Result 2 ---------------------------------------------
[[Business (100%)]] --> [[Sports (50%)]]

[[Carlyle]] Looks Toward Commercial Aerospace (Reuters) Reuters - [[Private]] [[investment]] firm [[Carlyle]] Group,\which has a reputation for making well-timed and occasionally\controversial plays in the defense [[industry]], has quietly placed\its bets on another part of the [[market]].

[[Carⅼyle]] Looks Toward Commercial Aerospace (Reuters) Reuters - [[Prіvate]] [[іnvestment]] firm [[Carlyⅼe]] Group,\which has a reputation for making well-timed and occasionally\controversial plays in the defense [[іndustry]], has quietly placed\its bets on another part of the [[marke𝚝]].




...

--------------------------------------------- Result 8 ---------------------------------------------
[[Business (100%)]] --> [[World (53%)]]

[[Fed]] [[minutes]] show dissent over inflation (USATODAY.com) USATODAY.com - [[Retail]] [[sales]] bounced back a bit in July, and new claims for jobless benefits fell last week, the government said Thursday, indicating the economy is improving from a midsummer slump.

[[Feԁ]] [[mіnutes]] show dissent over inflation (USATODAY.com) USATODAY.com - [[Retaіl]] [[saleѕ]] bounced back a bit in July, and new claims for jobless benefits fell last week, the government said Thursday, indicating the economy is improving from a midsummer slump.




[Succeeded / Failed / Skipped / Total] 9 / 0 / 0 / 9:  90%|█████████ | 9/10 [15:27<01:43, 103.08s/it]

--------------------------------------------- Result 9 ---------------------------------------------
[[Business (100%)]] --> [[Sci/tech (95%)]]

Safety Net ([[Forbes]].com) [[Forbes]].com - After earning a PH.D. in Sociology, Danny Bazil Riley started to work as the general manager at a commercial real estate firm at an annual base salary of  #36;70,000. Soon after, a financial planner stopped by his desk to drop off brochures about insurance benefits available through his employer. But, at 32, "buying insurance was the furthest thing from my mind," says Riley.

Safety Net ([[ForЬes]].com) [[Fоrbes]].com - After earning a PH.D. in Sociology, Danny Bazil Riley started to work as the general manager at a commercial real estate firm at an annual base salary of  #36;70,000. Soon after, a financial planner stopped by his desk to drop off brochures about insurance benefits available through his employer. But, at 32, "buying insurance was the furthest thing from my mind," says Riley.




[Succeeded / Failed / Skipped / Total] 10 / 0 / 0 / 10: 100%|██████████| 10/10 [16:09<00:00, 96.91s/it]

--------------------------------------------- Result 10 ---------------------------------------------
[[Business (100%)]] --> [[World (99%)]]

Wall [[St]]. Bears Claw Back Into the Black  NEW YORK (Reuters) - [[Short-sellers]], Wall [[Street's]] dwindling  band of ultra-cynics, are seeing green again.

Wall [[S𝚝]]. Bears Claw Back Into the Black  NEW YORK (Reuters) - [[Short˗sellers]], Wall [[S𝚝reet's]] dwindling  band of ultra-cynics, are seeing green again.



+-------------------------------+--------+
| Attack Results                |        |
+-------------------------------+--------+
| Number of successful attacks: | 10     |
| Number of failed attacks:     | 0      |
| Number of skipped attacks:    | 0      |
| Original accuracy:            | 100.0% |
| Accuracy under attack:        | 0.0%   |
| Attack success rate:          | 100.0% |
| Average perturbed word %:     | 9.55%  |
| Average num. words per input: | 38.5   |
| Avg num queries:              | 2804.4 |
+----------------